In [24]:
#Web scraping data from mohfw
import requests #this library will help to open the HTML page in front of us and show it to us that is request is sent to open up the URL
import pandas as pd #for data manipulation
import numpy as np
from bs4 import BeautifulSoup

url = 'https://www.mohfw.gov.in/'# make a GET request to fetch the raw HTML content
web_content = requests.get(url).content# parse the html content
soup = BeautifulSoup(web_content, "html.parser")

# remove any newlines and extra spaces from left and right
extract_contents = lambda row: [x.text.replace('\n', '') for x in row]

stats = [] 
all_rows = soup.find_all('tr')

for row in all_rows:
    stat = extract_contents(row.find_all('td')) # notice that the data that we require is now a list of length 5
    if len(stat) == 5:
        stats.append(stat)#now convert the data into a pandas dataframe for further processing
new_cols = ["Sr.No", "States/UT","Confirmed","Recovered","Deceased"]
state_data = pd.DataFrame(data = stats, columns = new_cols)

state_data.Confirmed = state_data.Confirmed.map(int)
state_data.Recovered = state_data.Recovered.map(int)
state_data.Deceased = state_data.Deceased.map(int)

# Nationwide Statistics on Novel Corona Virus 19

In [25]:
#want to show total stats
Total_confirmed = sum(state_data.Confirmed)
Total_deceased = sum(state_data.Deceased)
Total_recovered = sum(state_data.Recovered)
Total_active = Total_confirmed - (Total_deceased + Total_recovered)

from IPython.display import display, HTML, Image

# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(Total_confirmed) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(Total_deceased) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(Total_recovered) + "</span>"+
             "<span style='color: yellow; font-size:30px; margin-left:20px;'> Active: " + str(Total_active) + "</span>"+
             "</div>")
       )

In [26]:
#will show the entire table
state_data['Active'] = state_data.Confirmed - state_data.Recovered - state_data.Deceased

In [27]:
state_data.drop('Sr.No', inplace = True, axis = 1)

In [28]:
#import geopandas as gpd

import plotly.express as px
import plotly
from plotly import version
#print (version)
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)

import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected = True)

In [29]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore') 

# Statewise Breakdown

In [30]:
fig = go.FigureWidget( layout=go.Layout() )
def select_states(State):
    df = state_data
    if State == 'All':
        return df.style.set_properties(**{'background-color': 'aquamarine',
                            'color': 'black',
                            'border-color': 'white'})
    return df[df['States/UT'] == State].style.set_properties(**{'background-color': 'aquamarine',
                            'color': 'black',
                            'border-color': 'white'})

states_list = ['All']+sorted(state_data['States/UT'].unique().tolist())
w1 = widgets.Select(
    options=states_list,
    rows = 1,
    disabled=False,
    description = 'States'
)
interact(select_states, State=w1)
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Select(description='States', options=('All', 'Andaman and Nicobar Islands', 'Andhra Prad…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [828]:
#def select_states(State):
 #   fig = go.FigureWidget(data=[go.Table(
  #      header=dict(values=list(state_data.columns),
   #                 fill_color='darkgray',
    #                line_color = 'darkslategray',
     #               align=['left','center'],
      #             font=dict(color='white', size=12)),
       # cells=dict(values=[state_date[state_data['States/UT']==State], state_data.Confirmed, state_data.Deceased, state_data.Recovered,
        #                  state_data.Active],
         #          fill_color='chocolate',
          #         align=['left','center'],
           #       font = dict(color = 'white', size = 10)))
    #])
#fig.show()


# Worst Hit States in terms of Number of Confirmed Cases

In [196]:
#fig = go.FigureWidget( layout=go.Layout() )
#def top_confirmed(n):
 #   state_data_new = state_data.sort_values(by = 'Confirmed', ascending = False).head(n)
  #  data = [go.Bar(x = state_data_new['States/UT'], y = state_data_new['Confirmed'])]#, color= 'Confirmed')
   # layout = go.Layout(
        #xaxis = dict(title = 'Website'),
        #yaxis = dict(title = 'Exception count'),
    #    title = f'Top {n} worst hit states')
    #chart = go.Figure(data = data, layout = layout)
    #py.iplot(chart, filename = 'WebExceptions')
    #pyo.plot(chart)
#widgets.interact(top_confirmed, n = (1,36))
#ipywLayout = widgets.Layout(border='solid 2px green')
#ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
#widgets.VBox([fig], layout=ipywLayout)

<function __main__.top_confirmed(n)>

In [31]:
fig = go.FigureWidget( layout=go.Layout() )
def top_confirmed(n):
    state_data_new = state_data.sort_values(by = 'Confirmed', ascending = False)
    fig = px.bar(state_data_new.head(n), x = 'States/UT', y = 'Confirmed', color_discrete_sequence=["darkgreen"])
    fig.update_layout(hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"))
    fig.show()
widgets.interact(top_confirmed, n = 10)
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Worst Hit States in terms of Number of Deaths

In [32]:
fig = go.FigureWidget( layout=go.Layout() )
def top_deaths(n):
    state_data_new = state_data.sort_values(by = 'Deceased', ascending = False)
    fig = px.bar(state_data_new.head(n), x = 'States/UT', y = 'Deceased', color_discrete_sequence=["maroon"])
    fig.update_layout(hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"))
    fig.show()
widgets.interact(top_deaths, n = 10)
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Statewise Death to Confirmed Ratio

In [33]:
state_data_2 = state_data

In [34]:
state_data_2['Death to Confirmed Ratio'] = round(state_data.Deceased/state_data.Confirmed* 100,2)

In [35]:
fig = go.FigureWidget( layout=go.Layout() )
def top_ratio(n):
    state_data_ratio = state_data_2.sort_values(by = 'Death to Confirmed Ratio', ascending = False)
    fig = px.scatter(state_data_ratio.head(n), x = 'States/UT', y = 'Death to Confirmed Ratio', color = 'States/UT', size = 'Death to Confirmed Ratio',
                    hover_data= ['Confirmed','Deceased'], size_max = 60)
    fig.show()
widgets.interact(top_ratio, n = 10)
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [80]:
#for this taking data from John Hopkins site and filtering for India

In [36]:
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
country = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")

def melt_df(x, case_type):
    new_df = x.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'])
    new_df.rename(columns={'variable': 'Date', 'value':case_type}, inplace = True)
    return new_df

confirm_df = melt_df(confirmed, 'Confirmed')
death_df = melt_df(deaths, 'Deaths')
recovered_df = melt_df(recovered, 'Recovered')

def merge_df(confirm_df, death_df, recovered_df):
    new_df = confirm_df.join(recovered_df['Recovered']).join(death_df['Deaths'])
    return new_df


final_df = merge_df(confirm_df, death_df, recovered_df)

In [37]:
India = final_df[final_df['Country/Region'] == 'India']
#India

# Cumulative confirmed and death cases 

In [38]:
import warnings
warnings.filterwarnings('ignore') 

In [39]:
fig = go.FigureWidget()
fig.add_trace(go.Scatter(x=India.Date, y=India.Confirmed,
                         line=dict(color='darkgreen', width=2), mode = 'lines+markers',
                        name = 'Confirmed'))
fig.add_trace(go.Scatter(x=India.Date, y=India.Deaths,
                         line=dict(color='red', width=2), mode = 'lines+markers',
                        name = 'Deaths'))


FigureWidget({
    'data': [{'line': {'color': 'darkgreen', 'width': 2},
              'mode': 'lines+markers'…

# Daily confirmed and death cases

In [40]:
India['Daily New Cases'] = India.Confirmed.diff()
India['Daily New Deaths'] = India.Deaths.diff()

In [41]:
fig = go.FigureWidget()
fig.add_trace(go.Scatter(x=India.Date, y=India['Daily New Cases'],
                         line=dict(color='darkgreen', width=2), mode = 'lines+markers',
                        name = 'New Cases'))
fig.add_trace(go.Scatter(x=India.Date, y=India['Daily New Deaths'],
                         line=dict(color='red', width=2), mode = 'lines+markers', name = 'New Deaths'))
#fig.show()

FigureWidget({
    'data': [{'line': {'color': 'darkgreen', 'width': 2},
              'mode': 'lines+markers'…

In [42]:
state_data_new = state_data.copy()
state_data_new['States/UT'] = state_data_new['States/UT'].str.replace('#', '')

In [43]:
#creating a dataframe with lat long and state name
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 'Maharashtra':19.7515,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670}

long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662}

In [44]:
state_data_new['Latitude'] = state_data_new['States/UT'].map(lat)
state_data_new['Longitude'] = state_data_new['States/UT'].map(long)

In [45]:
import folium
from IPython.display import HTML, display

# Zoom In to India see the statewise stats

In [46]:
India_map = folium.Map(location=[21,-78], tiles="cartodbpositron", zoom_start=2, max_zoom = 60, min_zoom = 2)

for i in range(0,len(state_data_new)):
    folium.Circle(
        location=[state_data_new.iloc[i]['Latitude'], state_data_new.iloc[i]['Longitude']],
        fill=True,
        radius=(int((np.log(state_data_new.iloc[i,1]+1.00001)))+0.2)*10000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+state_data_new.iloc[i]['States/UT'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(state_data_new.iloc[i,1])+"</li>"+
                        "<li>Deaths:   "+str(state_data_new.iloc[i,3])+"</li>"+
                        "<li>Percentage Cases: "+str(round(state_data_new.iloc[i,1]/sum(state_data_new.iloc[:,1])*100,1))+ str("%") + "</li>"+
                    "</ul></div>",
        ).add_to(India_map)
India_map

In [ ]:
# we will use binder to deploy our notebook to the web
#binder turns a siple github repo into an interactive notebook environment